## this file use to predict the image

In [2]:

#%tensorflow_version 2.x
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow_addons as tfa
import nibabel as nib
from pathlib import Path
import cv2
from model import unet
import glob
import json
DATA_DIR = Path("./raw")
IMG_DIR_NAME = "img/ICH"
SI_IMG_DIR_NAME = "img/SI"
MASK_DIR_NAME = "mask"
IMG_DIR = str(DATA_DIR / IMG_DIR_NAME)
SI_IMG_DIR = str(DATA_DIR / SI_IMG_DIR_NAME)
MODEL_GRAVEYARD = Path("./model_saved/channel_seperated_version")
IMG_GRAVEYARD = Path("./saved_img") 

INPUT_SIZE = OUTPUT_SIZE = MASK_SIZE = (224, 224) 

OUTPUT_CHANNELS = 4
MASK_SHAPE = OUTPUT_SHAPE = (*OUTPUT_SIZE, OUTPUT_CHANNELS) # (224, 224, 3)
INTERMEDIATE_SIZE = (250, 250)

PIXEL_THRESHOLD = 100
BATCH_SIZE = 8
BUFFER_SIZE = 128
TRAIN_LENGTH = 2166
TEST_LENGTH = 1000
VAL_SUBSPLITS = 5
VALIDATION_STEPS = TEST_LENGTH // BATCH_SIZE // VAL_SUBSPLITS
# STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE
STEPS_PER_EPOCH = 5
MIN_SHIFTING = -5
MAX_SHIFTING = 6
RANDOM_RADIAN_MAX = 15.0 * np.pi / 180
FOLDS = 5
MIN_INPUT_VALUE = 0
MAX_INPUT_VALUE = 1

MAP_COLORS = np.array(
    [
        [  0,   0,   0], # black / layer
        # [215, 175, 168], # brownish pink
        [ 56, 180, 139],  # green 
        [ 50,  76, 172], # dark blue
        [ 66, 255, 255], # cyan
     
    ], dtype="uint8")
# from segmentation_models import Unet as unet
WINDOWING_MIN_MAX = np.array(((0, 40), (0, 20),(20, 40)))

INPUT_SHAPE = (224, 224, len(WINDOWING_MIN_MAX))

In [3]:
def class_wise_metrics(y_true, y_pred):
    class_wise_iou = []
    class_wise_dice_score = []

    smoothening_factor = 0.00001
    for i in range(OUTPUT_CHANNELS):

        intersection = np.sum((y_pred == i) * (y_true == i))
        y_true_area = np.sum((y_true == i))
        y_pred_area = np.sum((y_pred == i))
        combined_area = y_true_area + y_pred_area

        iou = (intersection + smoothening_factor) / \
            (combined_area - intersection + smoothening_factor)
        class_wise_iou.append(iou)

        dice_score = 2 * ((intersection + smoothening_factor) /
                          (combined_area + smoothening_factor))
        class_wise_dice_score.append(dice_score)

    return class_wise_iou, class_wise_dice_score


In [4]:
def window_ct_v3(data_array, w_min, w_max, min_value=0, max_value=255, dtype="uint8"):
    if "float" in dtype:  # division is problem
        data_array = tf.cast(data_array, dtype)
    data_array = ((data_array - w_min) / (w_max - w_min)
                  * (max_value - min_value)) + min_value
    data_array = tf.where(data_array < min_value, min_value, data_array)
    data_array = tf.where(data_array > max_value, max_value, data_array)
    return tf.cast(data_array, dtype)


In [5]:
def foldSI():
    subject = os.listdir(SI_IMG_DIR)
    subject_id = {}
    for i in subject:
        tmp_id = i.split('-')[0][1:]
        if subject_id.get(tmp_id):
            subject_id[tmp_id] += 1
        else:
            subject_id[tmp_id] = 1
    folded_paths = []
    for i in subject_id.keys():
        subject_list = []
        for filename in subject:
            if i in filename:
                subject_list.append(filename)
    #     print(i,np.asarray(subject_list))
        folded_paths.append(np.asarray(subject_list))

    SI_folded_paths = []
    for i in range(FOLDS):
        tmp_SI_folded_paths = np.array([])
        for j in range(len(folded_paths)//FOLDS):
            subject_id = i*len(folded_paths)//FOLDS+j
            tmp_SI_folded_paths = np.concatenate(
                (folded_paths[subject_id], tmp_SI_folded_paths))

        SI_folded_paths.append(tmp_SI_folded_paths)
    folded_paths = SI_folded_paths
    return folded_paths


In [6]:
colors = [
    0,
    1,
    2,
    3
]


def create_one_hot_mask(mask, dtype):
    one_hot_map = []
    for color in colors:
        class_map = tf.reduce_all(tf.equal(mask, color), axis=-1)
        one_hot_map.append(class_map)

    one_hot_map = tf.stack(one_hot_map, axis=-1)
    one_hot_map = tf.cast(one_hot_map, dtype)
    return one_hot_map


def create_mask(one_hot_mask):
    """
    (width, height, n_classes) -> (width, height, 1)
    (batch, width, height, n_classes) -> (batch, width, height, 1)
    """
#     print(one_hot_mask,type(one_hot_mask))
#     one_hot_mask_int = tf.cast(one_hot_mask, tf.int32)
#     pred_mask = tf.argmax(one_hot_mask_int, axis=-1)
    pred_mask = tf.argmax(one_hot_mask, axis=-1)
    return pred_mask[..., tf.newaxis]


def create_visible_mask(gt, mask, is_one_hot=False):
    if is_one_hot:
        mask = create_mask(mask)
    if tf.reduce_max(gt) <= 1.0:  # for floating-point `gt`, looking for better way to do this
        gt = gt * 255
    gt = tf.cast(gt, tf.int16)
    for i in range(OUTPUT_CHANNELS):
        gt = tf.where(mask == i, gt + MAP_COLORS[i], gt)
    gt = tf.where(gt > 255, 255, gt)
    return gt


In [7]:
def create_generator_SI(train_or_test, fold_index=FOLDS-1, with_name=False,  augment=False):

    def parse_image():
        shifting_windowing = tf.zeros(shape=[WINDOWING_MIN_MAX.shape[0]])
        all_path = os.listdir(IMG_DIR)
        folded_paths = foldSI()
        tmp_IMG_DIR = 'raw/img/SI'
        tmp_IMG_DIR_NAME = 'img/SI'
        tmp_MASK_DIR_NAME = 'mask'

        if "test" in train_or_test:
            paths = folded_paths[fold_index]
            tmp_IMG_DIR = 'raw/img/SI'
            tmp_IMG_DIR_NAME = 'img/SI'
            tmp_MASK_DIR_NAME = 'mask'

        elif "SI_wo_norm" in train_or_test:
            paths = os.listdir('./convert_data')
            tmp_IMG_DIR = 'convert_data'
            tmp_IMG_DIR_NAME = './'
            tmp_MASK_DIR_NAME = 'mask'

        elif "train" in train_or_test:
            print('train')
            folded_paths.pop(fold_index)
            paths = []
            tmp_IMG_DIR = 'raw/img/SI'
            tmp_IMG_DIR_NAME = 'img/SI'
            tmp_MASK_DIR_NAME = 'mask'
            for p in folded_paths:
                for i in p:
                    paths.append(i)
        elif "SI" in train_or_test:
            paths = os.listdir('./raw/img')
            tmp_IMG_DIR = 'raw/img'
            tmp_IMG_DIR_NAME = 'img/SI'
            tmp_MASK_DIR_NAME = 'mask'
        else:
            paths = all_path

        for img_name in paths:
            img_path = tmp_IMG_DIR + "/" + img_name
            image = nib.load(img_path).get_fdata()
            mask_path = img_path.replace(tmp_IMG_DIR_NAME, tmp_MASK_DIR_NAME)
            mask = nib.load(mask_path).get_fdata()
            image = tf.convert_to_tensor(image)
            mask = tf.convert_to_tensor(mask)
            if augment and "train" in train_or_test:
                # augmentation include random crop, which will crop img to INPUT_SIZE, as well as mask
                image, mask = data_augmentation(image, mask)

            INPUT_SHAPE = (
                image.shape[0], image.shape[1], len(WINDOWING_MIN_MAX))

            for i in range(image.shape[2]):  # each slice
                # not really pixel count, but i think i fine to do it like this
                if tf.reduce_sum(mask[..., i]) > PIXEL_THRESHOLD:
                    if augment and "train" in train_or_test:
                        shifting_windowing = tf.random.uniform(shape=[WINDOWING_MIN_MAX.shape[0]],
                                                               minval=MIN_SHIFTING,
                                                               maxval=MAX_SHIFTING)
                    new_image = np.ones(INPUT_SHAPE, dtype="float32")
                    for j in range(INPUT_SHAPE[2]):
                        new_image[..., j] = window_ct_v3(image[..., i],
                                                         WINDOWING_MIN_MAX[j, 0] +
                                                         shifting_windowing[j],
                                                         WINDOWING_MIN_MAX[j, 1] +
                                                         shifting_windowing[j],
                                                         min_value=MIN_INPUT_VALUE,
                                                         max_value=MAX_INPUT_VALUE,
                                                         dtype="float32")

                    new_mask = create_one_hot_mask(
                        mask[..., i, tf.newaxis], tf.uint8)
                    new_image = tf.constant(new_image, dtype=tf.float32)
                    if with_name:
                        new_postfix = "_%d.png" % (i)
                        new_img_name = img_name.replace(".nii", new_postfix)
                        yield new_image, new_mask, [new_img_name]
                    else:
                        yield new_image, new_mask
    return parse_image


In [8]:
def create_generator_for_eval_only(train_or_test="", fold_index=FOLDS-1, with_name=False, augment=False):
    """
    img, [file_name]
    """
    print('eval')

    def parse_image():
        all_path = os.listdir(IMG_DIR)
        folded_paths = np.array_split(all_path, FOLDS)
        tmp_IMG_DIR = IMG_DIR
        if "test" in train_or_test:
            print("test")

            paths = folded_paths[fold_index]
        elif "train" in train_or_test:
            print("train")

            folded_paths.pop(fold_index)
            paths = []
            for p in folded_paths:
                for i in p:
                    paths.append(i)
        elif "SI" in train_or_test:
            print("si")
            paths = os.listdir(tmp_IMG_DIR)
        elif "test" in train_or_test:
            print("SI_norm")
            paths = os.listdir('./test')
            tmp_IMG_DIR = './test'
        elif "wo" in train_or_test:
            print("SI_wo_norm")
#             paths = os.listdir('./convert_data')
#             tmp_IMG_DIR = './convert_data'
            paths = os.listdir('./left')
            tmp_IMG_DIR = './left'
        else:
            print("else")
            paths = all_path

        for img_name in paths:
            img_path = tmp_IMG_DIR + "/" + img_name
            image = nib.load(img_path).get_fdata()
            print('origin ', image.shape, np.max(image))
            image = check_orientation(
                img_path, nib.load(img_path).affine, image)
            print('new ', image.shape, image.shape[2])
            INPUT_SHAPE = (
                image.shape[0], image.shape[1], len(WINDOWING_MIN_MAX))

            for i in range(image.shape[2]):  # each slice
                new_image = np.ones(INPUT_SHAPE, dtype="float32")
                for j in range(INPUT_SHAPE[2]):
                    new_image[..., j] = window_ct_v3(image[..., i],
                                                     WINDOWING_MIN_MAX[j, 0],
                                                     WINDOWING_MIN_MAX[j, 1],
                                                     min_value=MIN_INPUT_VALUE,
                                                     max_value=MAX_INPUT_VALUE,
                                                     #                                                      max_value=1.0,
                                                     dtype="float32")
                new_image = tf.constant(new_image, dtype=tf.float32)
                if with_name:
                    new_postfix = "_%d.png" % (i)
                    new_img_name = img_name.replace(".nii", new_postfix)
                    yield new_image, [new_img_name]
                else:
                    yield new_image
    return parse_image


In [9]:
def plot_mask_side_by_side(img, label=None, predicted_label=None, save_name=None, show=True, figsize=15, bone=False):
    row = 3
    c = 1
    INPUT_SHAPE = (img.shape[0], img.shape[1], len(WINDOWING_MIN_MAX))
    fig = plt.figure(figsize=(int(figsize*INPUT_SHAPE[2]/row), figsize))
    if predicted_label is None:
        row = 2
    img = tf.cast(img, tf.float32)
    img = window_ct_v3(img, tf.reduce_min(
        img), tf.reduce_max(img), 0, 1, "float32")

    for i in range(INPUT_SHAPE[2]):
        ax1 = fig.add_subplot(row, INPUT_SHAPE[2], c)
        ax1.set_title("gt%d w_min=%d, w_max=%d" %
                      (i, WINDOWING_MIN_MAX[i][0], WINDOWING_MIN_MAX[i][1]))
        ax1.imshow(img[..., i], cmap="gray")
        ax1.axis("off")
        c = c + 1

    if not label is None:
        for i in range(INPUT_SHAPE[2]):
            visible_mask = create_visible_mask(
                img[..., i][..., tf.newaxis], label, is_one_hot=True)
            ax4 = fig.add_subplot(row, INPUT_SHAPE[2], c)
            ax4.set_title("true mask on gt%d" % (i))
            ax4.imshow(visible_mask)
            ax4.axis("off")
            c = c + 1

    if not predicted_label is None:
        for i in range(INPUT_SHAPE[2]):
            visible_mask = create_visible_mask(
                img[..., i][..., tf.newaxis], predicted_label, is_one_hot=True)
            ax7 = fig.add_subplot(row, INPUT_SHAPE[2], c)
            ax7.set_title("predicted mask on gt%d" % (i))
            ax7.imshow(visible_mask)
            ax7.axis("off")
            c = c + 1

    if save_name:
        fig.savefig(str(save_name))
    if show:
        plt.show()
        print()
    fig.clear()
    plt.close(fig)


In [10]:
import scipy
import imageio


def create_generator_wo_pixel_count_ByTime(train_or_test="test", fold_index=FOLDS-1, with_name=False, augment=False):
    """
    img, [file_name]
    """
    def parse_image():
        tmp_IMG_DIR = IMG_DIR
        if "test" in train_or_test:
            train_test_folds = json.load(open('folds5_42.json'))
            idx_all = train_test_folds[str(fold_index)][train_or_test]
            paths = idx_all
            print(paths)
        elif "train" in train_or_test:
            train_test_folds = json.load(open('folds5_42.json'))
            idx_all = train_test_folds[str(fold_index)][train_or_test]
            if train_or_test == 'train':
                idx_all += train_test_folds['additional']
            paths = idx_all
            print(paths)
        elif "NPH" in train_or_test:
            tmp_IMG_DIR = './SI_dataset/NPH/selected'
            paths = os.listdir(tmp_IMG_DIR)
        elif "55" in train_or_test:
            tmp_IMG_DIR = './SI_dataset/NPH_Aug2022/s'
            paths = os.listdir(tmp_IMG_DIR)
        elif "revisited" in train_or_test:
            print('dataset_norm_mask_by_revisited')
            # Update the JSON file name
            train_test_folds = json.load(open('folds5_42_val.json'))
            paths = train_test_folds[str(fold_index)]["test"]

        elif "normal" in train_or_test:
            tmp_IMG_DIR = './SI_dataset/normal_mpr_non/non/selected'
            paths = os.listdir(tmp_IMG_DIR)

        elif "wo" in train_or_test:
            print("SI_wo_norm")
            paths = os.listdir('./convert_data')
            tmp_IMG_DIR = './convert_data'
        else:
            print("else")
            return

        for img_name in paths:
            img_path = img_name
            image = nib.load(img_path).get_fdata()
            image = tf.image.resize(image, INPUT_SIZE)

            for i in range(image.shape[2]):  # each slice
                new_image = np.ones(INPUT_SHAPE, dtype="float32")
                for j in range(INPUT_SHAPE[2]):
                    new_image[..., j] = window_ct_v3(image[..., i],
                                                     WINDOWING_MIN_MAX[j, 0],
                                                     WINDOWING_MIN_MAX[j, 1],
                                                     max_value=1.0,
                                                     dtype="float32")
                new_image = tf.constant(new_image, dtype=tf.float32)
                if with_name:
                    if 'test' in img_name:
                        new_postfix = "test_%d.png" % (i)
                    else:
                        new_postfix = "_%d.png" % (i)
                    new_img_name = img_name.replace(".nii", new_postfix)
                    yield new_image, [new_img_name]
                else:
                    yield new_image
    return parse_image


In [13]:
'''
for get predict mask from segmentation model in setting below use segmentation model fold 0
by name_model is selected segmentation model
'''
for i in [0]:
    name_model = f'42_focalloss_200epochs_lr_init=0.001_unet_scaled_(0,40)(0,20)(20,40)_224_allSlice_byTime_fold{i}'
    print(name_model)
    model = tf.keras.models.load_model(
        f'/data2-2tb/kgreathoo/model_saved/channel_seperated_version/{name_model}/best_model.h5', compile=False)
    ds2 = tf.data.Dataset.from_generator(create_generator_wo_pixel_count_ByTime("revisited", fold_index=i, with_name=True),
                                         output_signature=(tf.TensorSpec(shape=INPUT_SHAPE, dtype=tf.float32),
                                                           tf.TensorSpec(shape=(1, ), dtype=tf.string))
                                         )
    all_ds = ds2.batch(32)
    out_dir = IMG_GRAVEYARD/f'test2014_{name_model}'
    out_dir.mkdir(parents=True, exist_ok=True)
    all_iter = all_ds.as_numpy_iterator()
    out_shape = (*INPUT_SHAPE[0:2], 4)  # (224, 224, 3)
    not_end = True

    for gt_m, name_m in all_iter:
        pred_m = model(gt_m)
        for i in range(pred_m.shape[0]):
            #             plot_mask_side_by_side(gt_m[i],pred_m[i])
            mask_export = np.broadcast_to(
                create_mask(pred_m[i]).numpy(), shape=out_shape)
            out_dir_file = str(out_dir/name_m[i, 0].decode().split('/')[-1])
            cv2.imwrite(out_dir_file, mask_export)


42_focalloss_200epochs_lr_init=0.001_unet_scaled_(0,40)(0,20)(20,40)_224_allSlice_byTime_fold0
dataset_norm_mask_by_revisited


In [15]:
from auxiliary import load_nifti
import numpy as np
import nibabel as nib
import fnmatch
import os
import cv2
import re
from pathlib import Path
import glob
'''
use png mask from model to create nitfi mask
'''
def extract_brain_component_v1 (hu, lower_grey, upper_grey) :
    rows = hu.shape[0]
    cols = hu.shape[1]

    img =  np.zeros((rows,cols,3), "uint8")
    
    for i in range (0,rows) :
        for j in range (0,cols) :
            if (hu[i,j] > lower_grey) and (hu[i,j] < upper_grey):
                img[i,j] = (255,255,255) 

    return img

def extract_brain_component_v2(ct_scan, w_min, w_max):
    """ct_scan: np.ndarray
    w_level: int (hounsfield unit)
    w_width: int (hounsfield unit)
    """
    ct_scan = ct_scan[..., np.newaxis]
    mask = np.where((ct_scan > w_min) & (ct_scan < w_max), [255, 255, 255], [0, 0, 0])
    return mask


def create_nifti_from_png(png_dir: Path, patient_no: str, patient_rawdir: Path, out_dir: Path):
    raw_name = "%s.nii" % patient_no
    out_dir.mkdir(exist_ok=True)
    os.makedirs(out_dir, exist_ok=True)
    patient_raw = patient_rawdir
    file_name = None
    try:
        raw_array, raw_header = load_nifti(str(patient_raw)) #load nitfi file by patient
        new_mask = np.zeros(shape=(224, 224, raw_array.shape[2]), dtype="uint8")
        all_slice = fnmatch.filter(os.listdir(str(png_dir)), "%s*.*" % patient_no) #load png file by patient
        print(png_dir)
        if len(all_slice) == 0:
            raise FileNotFoundError()
        for file_name in all_slice:
            print("(%s)_(\\d+).*" % (patient_no), file_name)
            m = re.match("(%s)_(\\d+).*" % (patient_no), file_name) # find group of patient id
            if m is None:
                raise FileNotFoundError()
            i = int(m.group(2))
            try:
                new_mask[..., i] = cv2.imread(str(png_dir / file_name))[..., 0]
                print(i, end=" ", flush=True)
            except Exception as e:
                print(e)
            
        
        new_mask_file = out_dir / raw_name
        nib.save(nib.Nifti1Image(new_mask, affine=raw_header.affine, header=raw_header.header), str(new_mask_file)) 
        print()
    except FileNotFoundError:
        if file_name is not None:
            print("cant find file ", file_name)
            pass
        else:
            print("cant find id ", patient_no)
            pass
        

def create_nifti_whole_dir(png_dir, patient_rawdir, out_dir):
    for file_name in glob.glob('/data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/**/**/s/**.nii', recursive=True):
        f = file_name.split("/")[-1].split(".")[0] #extract patient no.
            
        patient_rawdir = file_name
        print('create_nii ',patient_rawdir,f)
        if not os.path.exists(out_dir / ("%s.nii" % f)):
            create_nifti_from_png(png_dir, f, Path(patient_rawdir), out_dir)
        


if __name__ == "__main__":
    IMG_GRAVEYARD = Path("./saved_img")
    for i in [0]:
        png_dir = Path(out_dir)
        patient_rawdir = Path("./SI_dataset/normal_mpr_non/non/nii")
        out_dir = Path(f'./saved_img/nii/check_42_old_{i}')
        os.makedirs(out_dir, exist_ok=True)

        create_nifti_whole_dir(png_dir, patient_rawdir, out_dir)

create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/normal/110/s/s42054989-0301-00001-000001.nii s42054989-0301-00001-000001
saved_img/nii/check_42_old_0
cant find id  s42054989-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/normal/110/s/s54210092-0301-00001-000001.nii s54210092-0301-00001-000001
saved_img/nii/check_42_old_0
cant find id  s54210092-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/normal/110/s/s46023027-0301-00001-000001.nii s46023027-0301-00001-000001
saved_img/nii/check_42_old_0
cant find id  s46023027-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/normal/110/s/s38182610-0301-00001-000001.nii s38182610-0301-00001-000001
saved_img/nii/check_42_old_0
cant find id  s38182610-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/normal/110/s/s50157980-0301-00001-000001.nii s50157980-0301-00001-000001
saved_img/nii/check_42_old_0
ca

saved_img/nii/check_42_old_0
cant find id  s44096626-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/normal/110/s/s42106198-0301-00001-000001.nii s42106198-0301-00001-000001
saved_img/nii/check_42_old_0
cant find id  s42106198-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/normal/110/s/s54263191-0301-00001-000001.nii s54263191-0301-00001-000001
saved_img/nii/check_42_old_0
cant find id  s54263191-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/normal/110/s/s53798105-0301-00001-000001.nii s53798105-0301-00001-000001
saved_img/nii/check_42_old_0
cant find id  s53798105-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/normal/110/s/s52501006-0301-00001-000001.nii s52501006-0301-00001-000001
saved_img/nii/check_42_old_0
cant find id  s52501006-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/normal/110/s/s46010936-0301-00001

saved_img/nii/check_42_old_0
cant find id  s44119324-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/normal/110/s/s54231068-0301-00001-000001.nii s54231068-0301-00001-000001
saved_img/nii/check_42_old_0
cant find id  s54231068-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/normal/110/s/s53851833-0301-00001-000001.nii s53851833-0301-00001-000001
saved_img/nii/check_42_old_0
cant find id  s53851833-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/normal/110/s/s54197829-0301-00001-000001.nii s54197829-0301-00001-000001
saved_img/nii/check_42_old_0
cant find id  s54197829-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/normal/110/s/s50501145-0003-00001-000001.nii s50501145-0003-00001-000001
saved_img/nii/check_42_old_0
cant find id  s50501145-0003-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/normal/110/s/s46501836-0301-00001

saved_img/nii/check_42_old_0
cant find id  s40028892-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/2norm59_oct_seg/s/s54097214-0301-00001-000001.nii s54097214-0301-00001-000001
saved_img/nii/check_42_old_0
cant find id  s54097214-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/2norm59_oct_seg/s/s41069055-0301-00001-000001.nii s41069055-0301-00001-000001
saved_img/nii/check_42_old_0
cant find id  s41069055-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/2norm59_oct_seg/s/s39009599-0301-00001-000001.nii s39009599-0301-00001-000001
saved_img/nii/check_42_old_0
cant find id  s39009599-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/2norm59_oct_seg/s/s53924279-0301-00001-000001.nii s53924279-0301-00001-000001
saved_img/nii/check_42_old_0
cant find id  s53924279-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisit

saved_img/nii/check_42_old_0
cant find id  s54128522-0304-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/2norm59_oct_seg/s/s54129096-0301-00001-000001.nii s54129096-0301-00001-000001
saved_img/nii/check_42_old_0
cant find id  s54129096-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/2norm59_oct_seg/s/s54128315-0301-00001-000001.nii s54128315-0301-00001-000001
saved_img/nii/check_42_old_0
cant find id  s54128315-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/2norm59_oct_seg/s/s38055905-0301-00001-000001.nii s38055905-0301-00001-000001
saved_img/nii/check_42_old_0
cant find id  s38055905-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/2norm59_oct_seg/s/s53794645-0301-00001-000001.nii s53794645-0301-00001-000001
saved_img/nii/check_42_old_0
cant find id  s53794645-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisit

...

saved_img/nii/check_42_old_0
cant find id  s38032221-0002-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/3nii_norm68/s/s38010532-0301-00001-000001.nii s38010532-0301-00001-000001
saved_img/nii/check_42_old_0
cant find id  s38010532-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/3nii_norm68/s/s38015557-0002-00001-000001.nii s38015557-0002-00001-000001
saved_img/nii/check_42_old_0
cant find id  s38015557-0002-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/3nii_norm68/s/s38027567-0002-00001-000001.nii s38027567-0002-00001-000001
saved_img/nii/check_42_old_0
cant find id  s38027567-0002-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/3nii_norm68/s/s40071790-0003-00001-000001.nii s40071790-0003-00001-000001
saved_img/nii/check_42_old_0
cant find id  s40071790-0003-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/3nii_norm

saved_img/nii/check_42_old_0
cant find id  s45109872-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/norm125/s/s45084446-0002-00001-000001_pre_fix_ge_ct.nii s45084446-0002-00001-000001_pre_fix_ge_ct
saved_img/nii/check_42_old_0
cant find id  s45084446-0002-00001-000001_pre_fix_ge_ct
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/norm125/s/s53966383-0002-00001-000001_pre_fix_ge_ct.nii s53966383-0002-00001-000001_pre_fix_ge_ct
saved_img/nii/check_42_old_0
cant find id  s53966383-0002-00001-000001_pre_fix_ge_ct
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/norm125/s/s53427016-0002-00001-000001_pre_fix_ge_ct.nii s53427016-0002-00001-000001_pre_fix_ge_ct
saved_img/nii/check_42_old_0
cant find id  s53427016-0002-00001-000001_pre_fix_ge_ct
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/norm125/s/s52589797-0002-00001-000001_pre_fix_ge_ct.nii s52589797-0002-00001-000001_pre_fix_ge_ct
saved_img/

saved_img/nii/check_42_old_0
cant find id  s38029223-0002-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/norm125/s/s48213983-0301-00001-000001.nii s48213983-0301-00001-000001
saved_img/nii/check_42_old_0
cant find id  s48213983-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/norm125/s/s41155224-0301-00001-000001_pre_fix_ge_ct.nii s41155224-0301-00001-000001_pre_fix_ge_ct
saved_img/nii/check_42_old_0
cant find id  s41155224-0301-00001-000001_pre_fix_ge_ct
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/norm125/s/s40038122-0301-00001-000001_pre_fix_ge_ct.nii s40038122-0301-00001-000001_pre_fix_ge_ct
saved_img/nii/check_42_old_0
cant find id  s40038122-0301-00001-000001_pre_fix_ge_ct
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/norm125/s/s44031522-0301-00001-000001_pre_fix_ge_ct.nii s44031522-0301-00001-000001_pre_fix_ge_ct
saved_img/nii/check_42_old_0
cant find id  s44031522

saved_img/nii/check_42_old_0
cant find id  s47121487-0002-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/norm125/s/s43127044-0301-00001-000001.nii s43127044-0301-00001-000001
saved_img/nii/check_42_old_0
cant find id  s43127044-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/norm125/s/s50193940-0002-00001-000001_pre_fix_ge_ct.nii s50193940-0002-00001-000001_pre_fix_ge_ct
saved_img/nii/check_42_old_0
cant find id  s50193940-0002-00001-000001_pre_fix_ge_ct
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/norm125/s/s53111503-0002-00001-000001_pre_fix_ge_ct.nii s53111503-0002-00001-000001_pre_fix_ge_ct
saved_img/nii/check_42_old_0
cant find id  s53111503-0002-00001-000001_pre_fix_ge_ct
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/norm125/s/s53885058-0301-00001-000001_pre_fix_ge_ct.nii s53885058-0301-00001-000001_pre_fix_ge_ct
saved_img/nii/check_42_old_0
cant find id  s53885058

saved_img/nii/check_42_old_0
cant find id  s49182781-0002-00001-000001_pre_fix_ge_ct
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/norm125/s/s50296397-0002-00001-000001.nii s50296397-0002-00001-000001
saved_img/nii/check_42_old_0
cant find id  s50296397-0002-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/norm125/s/s46025187-0002-00001-000001_pre_fix_ge_ct.nii s46025187-0002-00001-000001_pre_fix_ge_ct
saved_img/nii/check_42_old_0
cant find id  s46025187-0002-00001-000001_pre_fix_ge_ct
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/norm125/s/s38058123-0301-00001-000001.nii s38058123-0301-00001-000001
saved_img/nii/check_42_old_0
cant find id  s38058123-0301-00001-000001
create_nii  /data4-4tb/kgreathoo/dataset_norm_mask_by_revisited/nph/norm125/s/s39184244-0002-00001-000001_pre_fix_ge_ct.nii s39184244-0002-00001-000001_pre_fix_ge_ct
saved_img/nii/check_42_old_0
cant find id  s39184244-0002-00001-000001_pre_fix_g